In [9]:
import pandas as pd
import csv

In [10]:
def vocab_check(cond, vocab):
    cond_checked = []
    for word in cond:
        if word in vocab:
            cond_checked.append(vocab[word])
    cond_checked = [x for l in cond_checked for x in l]
    return cond_checked

In [16]:
data_proc = pd.read_csv('data_temp2.csv').drop(['index','Unnamed: 0'], axis=1)
data_proc['conditions_doc'] = data_proc['conditions_doc'].str.strip('[]').str.replace(' ', '').str.split(',')

In [17]:
data_proc

,rcsb_id,resolution,matthews,percent_solvent,pH,organism,sequence,conditions,conditions_doc
0,101M,2.07,3.09,60.20,9.0,Physeter catodon,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,"3.0 m ammonium sulfate, 20 mm tris, 1mm edta, ...","['3.0', 'm', 'ammonium', 'sulfate, ', '20', 'm..."
1,102M,1.84,3.09,60.20,9.0,Physeter catodon,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,"3.0 m ammonium sulfate, 20 mm tris, 1mm edta, ...","['3.0', 'm', 'ammonium', 'sulfate, ', '20', 'm..."
2,103M,2.07,3.09,60.30,9.0,Physeter catodon,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,"3.0 m ammonium sulfate, 20 mm tris, 1mm edta, ...","['3.0', 'm', 'ammonium', 'sulfate, ', '20', 'm..."
3,104M,1.71,1.87,34.30,7.0,Physeter catodon,VLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRF...,"3.0 m ammonium sulfate, unbuffered, ph 7.0","['3.0', 'm', 'ammonium', 'sulfate, ', 'unbuffe..."
4,105M,2.02,1.83,33.00,9.0,Physeter catodon,VLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRF...,"3.0 m ammonium sulfate, unbuffered, ph 9.0","['3.0', 'm', 'ammonium', 'sulfate, ', 'unbuffe..."
...,...,...,...,...,...,...,...,...,...
111998,9GSS,1.97,2.56,51.96,6.0,Homo sapiens,PPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKAS...,ph 6.0,"['ph', '6.0']"
111999,9ILB,2.28,3.36,60.00,8.5,Homo sapiens,APVRSLNCTLRDSQQKSLVMSGPYELKALHLQGQDMEQQVVFSMSF...,ph 8.5,"['ph', '8.5']"
112000,9JDW,2.50,3.83,68.00,7.0,Homo sapiens,STQAATASSRNSCAADDKATEPLPKDCPVSSYNEWDPLEEVIVGRA...,ph 7.0,"['ph', '7.0']"
112001,9MSI,2.60,2.10,41.56,5.5,Zoarces americanus,ANQASVVANQLIPINTALNLVMMRSEVVTPVGIPAEDIPRLVSMQV...,ph 5.5,"['ph', '5.5']"


In [13]:
with open('vocab_annotated.csv', mode = 'r') as infile:
    reader = csv.reader(infile)
    data_vocab = {rows[0]:rows[1] for rows in reader}
for key in data_vocab:
    data_vocab[key] = data_vocab[key].replace(' ','').split(',')


In [14]:
conditions_clean = []
for index, row in data_proc.iterrows():
    conditions_clean.append(vocab_check(row['conditions_doc'], data_vocab))
data_proc['conditions_clean'] = conditions_clean
data_proc['conditions_clean'] = data_proc['conditions_clean'].apply(lambda x: frozenset(x))

In [15]:
data_proc2 = data_proc.drop(labels=['conditions_doc'], axis=1)
data_proc2 = data_proc2[data_proc2['conditions_clean'] != frozenset() ].set_index('rcsb_id')
data_proc2['dupes'] = data_proc2.duplicated(['sequence', 'conditions_clean'])
data_clean = data_proc2[data_proc2['dupes'] == False].drop('dupes', axis=1)